## 1. Execute full pipeline

In [1]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, NB_TRIALS,
    OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.change_point_detector import ChangePointCostFunction, ChangePointMethod, get_change_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
print("Available devices:", tf.config.list_physical_devices())
print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
if tf.config.list_physical_devices("GPU"):
    try:
        tf.config.set_visible_devices([], "GPU")  # Hide traditional GPUs if any (like external ones)
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)
        print("✅ TensorFlow is using Apple GPU via MPS")
    except Exception as e:
        print("⚠️ Error setting MPS GPU:", e)
else:
    print("❌ No GPU found, running on CPU")

✅ TensorFlow is using Apple GPU via MPS


In [4]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'AIR_QUALITY'
change_point_method_argv = 'Window'
change_point_cost_function_argv = 'L1'

In [5]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{change_point_method_argv}_{change_point_cost_function_argv}_{SEED}"
change_point_method = ChangePointMethod.from_str(change_point_method_argv)
change_point_cost_function = ChangePointCostFunction.from_str(change_point_cost_function_argv)
change_point_approach = f"{change_point_method.value.title()} {change_point_cost_function.value.title()}"

In [6]:
print(f"[Step 1] Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

[Step 1] Reading dataset AIR_QUALITY from UCI
Variables: ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH']


In [7]:
print("[Step 2] Splitting data into train_val and test")
train_val, test = split_train_test(df)
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'change_point_method': change_point_method.value,
    'change_point_cost_function': change_point_cost_function.value,
    'change_point_approach': change_point_approach,
    'seed': SEED,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_val_shape': train_val.shape,
    'test_shape': test.shape,
}

[Step 2] Splitting data into train_val and test


In [8]:
print(f"[Step 3] Detecting cut point ({change_point_approach})")
start_time = time.time()
change_point_detector = get_change_point_detector(change_point_method, change_point_cost_function)
change_point, change_point_perc = change_point_detector.find_change_point(train_val, variables)
end_time = time.time()
detect_change_point_duration = end_time - start_time
print(f"Change point: {change_point}, Change point percentage: {change_point_perc}")
report.update({
    'detect_change_point_duration': detect_change_point_duration,
    'change_point': str(change_point),
    'change_point_perc': change_point_perc
})

[Step 3] Detecting cut point (Window L1)
Change point: 5385, Change point percentage: 71.9438877755511


In [9]:
print("[Step 4] Reducing train_val based on change point")
start_time = time.time()
reduced_train_val = change_point_detector.apply_change_point(train_val, change_point)
end_time = time.time()
apply_change_point_duration = end_time - start_time
report.update({
    'apply_change_point_duration': apply_change_point_duration,
    'reduced_train_val.shape': reduced_train_val.shape,
})

[Step 4] Reducing train_val based on change point


In [10]:
print("[Step 5] Splitting train_val into train and val")
reduced_train, reduced_val = split_train_test(reduced_train_val)
report.update({
    'reduced_train.shape': reduced_train.shape,
    'reduced_val.shape': reduced_val.shape,
})

[Step 5] Splitting train_val into train and val


In [11]:
print("[Step 6] Fitting scaler on train and applying on train and val")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_reduced_val = scaler.scale(reduced_val)
end_time = time.time()
fit_apply_scaler_train_val_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_duration': fit_apply_scaler_train_val_duration,
})

[Step 6] Fitting scaler on train and applying on train and val


In [12]:
print("[Step 7] Splitting train and val into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_reduced_scaled_val, y_reduced_scaled_val = split_X_y(scaled_reduced_val)
report.update({
    'X_reduced_scaled_train.shape': X_reduced_scaled_train.shape,
    'y_reduced_scaled_train.shape': y_reduced_scaled_train.shape,
    'X_reduced_scaled_val.shape': X_reduced_scaled_val.shape,
    'y_reduced_scaled_val.shape': y_reduced_scaled_val.shape,
})

[Step 7] Splitting train and val into X and y


In [13]:
y_reduced_scaled_train[0]

array([[ 0.2794384 ,  0.94363348, -0.22221197, -0.31901199,  2.57306525,
        -0.60601449],
       [ 0.93656296,  1.80147318,  0.20238044,  0.04013744,  2.3587538 ,
        -0.29619734],
       [ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888]])

In [14]:
y_reduced_scaled_train[1]

array([[ 0.93656296,  1.80147318,  0.20238044,  0.04013744,  2.3587538 ,
        -0.29619734],
       [ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888],
       [-0.26816541,  0.09679173, -0.64680438, -0.88942579,  1.46254228,
         0.52352721]])

In [15]:
y_reduced_scaled_train[2]

array([[ 1.81272904,  2.47234781,  0.6914199 ,  0.08239032,  2.20289093,
        -0.06383447],
       [ 1.26512524,  1.66949784,  0.35022958, -0.19225337,  2.12495949,
        -0.10901614],
       [ 0.88180258,  1.38355127,  0.17205241, -0.25563268,  1.91064804,
         0.1168922 ],
       [ 0.88180258,  1.30656566,  0.25924549, -0.53027636,  1.67685373,
         0.43316387],
       [ 0.8270422 ,  1.08660677,  0.17963441, -0.53027636,  1.52099086,
         0.56870888],
       [-0.26816541,  0.09679173, -0.64680438, -0.88942579,  1.46254228,
         0.52352721],
       [-0.76100883, -0.48609934, -1.01453173, -1.1218166 ,  1.44305942,
         0.52998174]])

In [16]:
print("[Step 8] Running HPO and NAS")
n_variables = len(variables)
forecaster_hypermodel = TimeSeriesHyperModel(
    n_variables=n_variables
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory=f"outputs/tuner/delete_me",
    project_name="delete_me",
    seed=SEED,
    overwrite=True,
    distribution_strategy=tf.distribute.MirroredStrategy()
)
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_data=(X_reduced_scaled_val, y_reduced_scaled_val),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time
report.update({
    'tuner_duration': tuner_duration
})

Trial 3 Complete [00h 00m 26s]
val_loss: 0.5227316369613012

Best val_loss So Far: 0.45366173021255
Total elapsed time: 00h 00m 49s


In [17]:
print("[Step 9] Retrieving best model")
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)
report.update({
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

[Step 9] Retrieving best model
Trial ID: 1
Hyperparameters: {'num_layers': 2, 'units_0': 128, 'learning_rate': 0.001, 'units_1': 96, 'units_2': 32, 'units_3': 64, 'batch_size': 48, 'epochs': 150}
Score: 0.45366173021255
----------------------------------------


/Users/zamith/Doutorado/TSForecastingSubsets/env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 14, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 96)             │        86,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 42)             │         4,074 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 6)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,490 (626.91 KB)

 Trainable params: 160,042 (625.16 KB)

 Non-trainable params: 448 (1.75 KB)

In [18]:
print("[Step 10] Fitting scaler on train_val and applying on train_val and test")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train_val = scaler.fit_scale(reduced_train_val)
scaled_test = scaler.scale(test)
end_time = time.time()
fit_apply_scaler_train_val_test_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_test_duration': fit_apply_scaler_train_val_test_duration,
})

[Step 10] Fitting scaler on train_val and applying on train_val and test


In [19]:
print("[Step 11] Splitting train_val and test into X and y")
X_reduced_scaled_train_val, y_reduced_scaled_train_val = split_X_y(scaled_reduced_train_val)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)
report.update({
    'X_reduced_scaled_train_val.shape': X_reduced_scaled_train_val.shape,
    'y_reduced_scaled_train_val.shape': y_reduced_scaled_train_val.shape,
    'X_scaled_test.shape': X_scaled_test.shape,
    'y_scaled_test.shape': y_scaled_test.shape,
})

[Step 11] Splitting train_val and test into X and y


In [20]:
print("[Step 12] Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train_val,
    y_reduced_scaled_train_val,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time
report.update({
    'retrain_duration': retrain_duration,
})

[Step 12] Retraining best model
Epoch 1/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.7836 - learning_rate: 0.0010
Epoch 2/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.7786 - learning_rate: 0.0010
Epoch 3/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.7422 - learning_rate: 0.0010
Epoch 4/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.7114 - learning_rate: 0.0010
Epoch 5/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.6611 - learning_rate: 0.0010
Epoch 6/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6358 - learning_rate: 0.0010
Epoch 7/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5979 - learning_rate: 0.0010
Epoch 8/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.5979 - learning_rate: 0.0010
Epoch 9/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5879 - learning_rate: 0.0010
Epoch 10/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6031 - learning_rate: 0.0010
Epoch 11/150
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - lo

In [21]:
print("[Step 13] Forecasting for test")
start_time = time.time()
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, n_variables)
y_scaled_pred_flat = y_scaled_pred.reshape(-1, n_variables)
end_time = time.time()
forecasting_test_duration = end_time - start_time
report.update({
    'forecasting_test_duration': forecasting_test_duration,
})

[Step 13] Forecasting for test


2025-03-11 13:35:32.217153: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

2025-03-11 13:35:32.570043: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [22]:
y_scaled_test_flat

array([[-1.23055822, -1.25086781, -1.16213415, -0.87719569, -2.06214374,
        -0.13780333],
       [-1.11462147, -1.18137868, -1.05524888, -0.6343508 , -1.94346707,
        -0.31248902],
       [-0.94071634, -0.99607432, -1.01566174, -0.52396676, -1.86434929,
        -0.3528011 ],
       ...,
       [-0.12915906,  0.06942571, -0.37435011,  1.24217789,  2.74426146,
        -2.84543152],
       [-0.3030642 , -0.26643843, -0.60395551,  0.82271854,  3.0211737 ,
        -3.16792817],
       [-0.24509582,  0.0115181 , -0.4851941 ,  1.08764024,  3.06073259,
        -3.1948029 ]])

In [23]:
y_scaled_pred_flat

array([[-1.2570964 , -1.118185  , -1.1714721 , -0.98450387, -0.8617839 ,
        -0.04249223],
       [-1.1576163 , -1.0291641 , -0.9263065 , -0.6987602 , -1.2499973 ,
        -0.23049735],
       [-0.8877931 , -0.56798494, -0.78847903, -0.58778214, -0.7765648 ,
        -0.69313884],
       ...,
       [ 0.22131073,  0.15229571,  0.10626084,  0.583874  ,  0.6346942 ,
        -0.6136454 ],
       [ 0.09056734,  0.2789734 ,  0.13814922,  0.42272183,  0.6354566 ,
        -0.821318  ],
       [ 0.29874888,  0.33242595,  0.179133  ,  0.48148185,  0.45168316,
        -0.9332367 ]], dtype=float32)

In [24]:
print("[Step 14] Descaling data")
start_time = time.time()
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))
end_time = time.time()
descaling_duration = end_time - start_time
report.update({
    'descaling_duration': descaling_duration,
})

[Step 14] Descaling data


In [25]:
y_test

,CO(GT),C6H6(GT),NOx(GT),NO2(GT),T,RH
0,0.5,1.0,94.0,79.0,2.6,58.6
1,0.7,1.6,121.0,90.0,3.2,56.0
2,1.0,3.2,131.0,95.0,3.6,55.4
3,3.0,15.9,552.0,169.0,1.8,63.3
4,4.1,18.0,614.0,199.0,2.4,60.2
...,...,...,...,...,...,...
12959,3.1,13.5,472.0,190.0,21.9,29.3
12960,2.4,11.4,353.0,179.0,24.3,23.7
12961,2.4,12.4,293.0,175.0,26.9,18.3
12962,2.1,9.5,235.0,156.0,28.3,13.5


In [26]:
y_pred

,CO(GT),C6H6(GT),NOx(GT),NO2(GT),T,RH
0,0.454219,2.145642,91.641174,74.139328,8.668723,60.018600
1,0.625831,2.914289,153.571793,87.082489,6.706013,57.220352
2,1.091297,6.896314,188.388000,92.109390,9.099570,50.334454
3,1.418148,7.119215,223.937363,91.017593,10.008091,51.046364
4,2.414800,5.756291,370.170319,109.554092,11.039125,51.086624
...,...,...,...,...,...,...
12959,2.737122,12.626399,522.500000,148.940964,14.510257,53.994621
12960,2.457779,13.777521,476.312988,145.011551,15.522722,48.495182
12961,3.004588,13.115537,414.405823,145.181198,16.234549,51.517628
12962,2.779045,14.209328,422.461060,137.881592,16.238403,48.426655


In [27]:
print("[Step 15] Calculating evaluation metrics")
total_duration = sum(value for key, value in report.items() if key.endswith('_duration'))
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")
report.update({
    'total_duration': total_duration,
    'error_results': error_results,
})

[Step 15] Calculating evaluation metrics
Obtained error results: {'Avg_MAPE': 4285350056737.7524, 'Avg_MAE': 37.014565271029056, 'Avg_MSE': 7450.922220362588, 'Avg_RMSE': 47.3275079827271, 'Avg_R2': 0.11642965684866236, 'Avg_WAPE': 0.4324391440854277, 'CO(GT)_MAPE': 1.21005619867109, 'CO(GT)_MAE': 1.0382033489363154, 'CO(GT)_MSE': 1.782331695345944, 'CO(GT)_RMSE': 1.335039960205665, 'CO(GT)_R2': 0.0715647930188531, 'CO(GT)_WAPE': 0.5468701850403123, 'C6H6(GT)_MAPE': 1.5977004653244824, 'C6H6(GT)_MAE': 5.071898450958814, 'C6H6(GT)_MSE': 40.501238878562354, 'C6H6(GT)_RMSE': 6.364058365427077, 'C6H6(GT)_R2': 0.031968018776954876, 'C6H6(GT)_WAPE': 0.6339057088573822, 'NOx(GT)_MAPE': 0.8774151298410675, 'NOx(GT)_MAE': 159.92815352222257, 'NOx(GT)_MSE': 41635.45162720768, 'NOx(GT)_RMSE': 204.0476699872059, 'NOx(GT)_R2': 0.04941531048386727, 'NOx(GT)_WAPE': 0.5309664025535077, 'NO2(GT)_MAPE': 0.30763528445930477, 'NO2(GT)_MAE': 40.33748309386435, 'NO2(GT)_MSE': 2801.0993754947763, 'NO2(GT)_RM

In [28]:
report

{'execution_id': 'validate_pipeline_UCI_AIR_QUALITY_Window_L1_42',
 'timestamp': 'validate_pipeline',
 'change_point_method': 'Window',
 'change_point_cost_function': 'L1',
 'change_point_approach': 'Window L1',
 'seed': 42,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'AIR_QUALITY',
 'variables': ['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH'],
 'dataset_shape': (9357, 7),
 'train_val_shape': (7485, 7),
 'test_shape': (1872, 7),
 'detect_change_point_duration': 0.07336115837097168,
 'change_point': '5385',
 'change_point_perc': 71.9438877755511,
 'apply_change_point_duration': 8.416175842285156e-05,
 'reduced_train_val.shape': (2100, 7),
 'reduced_train.shape': (1680, 7),
 'reduced_val.shape': (420, 7),
 'fit_apply_scaler_train_val_duration': 0.002841949462890625,
 'X_reduced_scaled_train.shape': (1660, 14, 6),
 'y_reduced_scaled_train.shape': (1660, 7, 6),
 'X_reduced_scaled_val.shape': (400, 14, 6),
 'y_reduced_scaled

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [29]:
X_train_val, y_train_val = split_X_y(train_val)
X_test, y_test = split_X_y(test)

In [30]:
train_val_targets_flat = pd.DataFrame(y_train_val.reshape(-1, len(variables)), columns=variables)
avg_values = train_val_targets_flat.mean(axis=0).to_numpy()

In [31]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, n_variables)
y_test_flat = pd.DataFrame(y_test.reshape(-1, n_variables), columns=variables)

In [32]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

Error metrics for Dummy Forecaster (predicting average values): 
{'Avg_MAPE': 8234635315137.397, 'Avg_MAE': 39.6761911395693, 'Avg_MSE': 9050.378952062862, 'Avg_RMSE': 54.45008211461132, 'Avg_R2': -0.6663866828639414, 'Avg_WAPE': 0.4635347737122664, 'CO(GT)_MAPE': 1.4907392486029176, 'CO(GT)_MAE': 1.1496764140739417, 'CO(GT)_MSE': 2.0059470835605415, 'CO(GT)_RMSE': 1.4163146131988265, 'CO(GT)_R2': -0.044918799672288845, 'CO(GT)_WAPE': 0.6055882539246805, 'C6H6(GT)_MAPE': 2.4158267445655204, 'C6H6(GT)_MAE': 5.998942145850846, 'C6H6(GT)_MSE': 49.37824329877635, 'C6H6(GT)_RMSE': 7.026965440271949, 'C6H6(GT)_R2': -0.18020386569282976, 'C6H6(GT)_WAPE': 0.7497712562918357, 'NOx(GT)_MAPE': 0.69058204879659, 'NOx(GT)_MAE': 152.8286100552228, 'NOx(GT)_MSE': 49209.25736669915, 'NOx(GT)_RMSE': 221.8315968627985, 'NOx(GT)_R2': -0.12350328403007538, 'NOx(GT)_WAPE': 0.5073956992631623, 'NO2(GT)_MAPE': 0.38503788600654404, 'NO2(GT)_MAE': 53.84588657162112, 'NO2(GT)_MSE': 4627.602712434949, 'NO2(GT)_R

In [33]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison

,Trained Model,Dummy Forecaster
Avg_MAPE,4.285350e+12,8.234635e+12
Avg_MAE,3.701457e+01,3.967619e+01
Avg_MSE,7.450922e+03,9.050379e+03
Avg_RMSE,4.732751e+01,5.445008e+01
Avg_R2,1.164300e-01,-6.663900e-01
Avg_WAPE,4.324400e-01,4.635300e-01
CO(GT)_MAPE,1.210060e+00,1.490740e+00
CO(GT)_MAE,1.038200e+00,1.149680e+00
CO(GT)_MSE,1.782330e+00,2.005950e+00
CO(GT)_RMSE,1.335040e+00,1.416310e+00
